# Manual Deployment of UUPS contracts for ownership by multisig vault

## Motivation
Tools like REMIX IDE and HardHat have plugins that automate and abstract the proxy contract creation and deployment.  This makes it hard to ensure that multisig vaults / wallets / contracts own the deployed assets.


In [1]:
import json
import web3

from web3 import Web3, HTTPProvider
from solcx import compile_standard, install_solc
from web3.contract import ConciseContract
from pathlib import Path

In [2]:
def get_contract_source(file_name):
    with open(file_name) as f:
        return f.read()

In [3]:
# py-solc-x documentation at https://solcx.readthedocs.io/en/latest/using-the-compiler.html
# good tutorial at https://coinsbench.com/how-to-compile-solidity-smart-contracts-with-python-98249cfc0879
# Install Solidity compiler.
_solc_version = "0.8.9"
install_solc(_solc_version)

Version('0.8.9')

In [4]:
oz_strings =  get_contract_source(Path('Contracts/openzeppelin/utils/Strings.sol'))
oz_context =  get_contract_source(Path('Contracts/openzeppelin/utils/Context.sol'))
oz_introspection_ierc165 =  get_contract_source(Path('Contracts/openzeppelin/utils/introspection/IERC165.sol'))
oz_introspection_erc165 =  get_contract_source(Path('Contracts/openzeppelin/utils/introspection/ERC165.sol'))
oz_i_access_control = get_contract_source(Path('Contracts/openzeppelin/access/IAccessControl.sol'))
oz_access_control = get_contract_source(Path('Contracts/openzeppelin/access/AccessControl.sol'))
oz_address = get_contract_source(Path('Contracts/openzeppelin/utils/Address.sol'))
oz_initializable = get_contract_source(Path('Contracts/openzeppelin/proxy/utils/Initializable.sol'))
oz_uups_upgradeable = get_contract_source(Path('Contracts/openzeppelin/proxy/utils/UUPSUpgradeable.sol'))
oz_ierc1822 = get_contract_source(Path('Contracts/openzeppelin/interfaces/draft-IERC1822.sol'))
oz_erc1967upgrade = get_contract_source(Path('Contracts/openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol'))
oz_beacon_ibeacon = get_contract_source(Path('Contracts/openzeppelin/proxy/beacon/IBeacon.sol'))
oz_storage_slot = get_contract_source(Path('Contracts/openzeppelin/utils/StorageSlot.sol'))

sabot_staking = get_contract_source(Path('Contracts/SabotStakingV1.sol'))



In [5]:
# compile the logic contract
# Compile SimpleStorage smart contract with solcx.


logic_contract = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            'openzeppelin/utils/Strings.sol':{"content": oz_strings},
            'openzeppelin/utils/Context.sol':{"content": oz_context},
            'openzeppelin/utils/introspection/IERC165.sol':{"content": oz_introspection_ierc165},
            'openzeppelin/utils/introspection/ERC165.sol':{"content": oz_introspection_erc165},
            'openzeppelin/access/IAccessControl.sol':{"content": oz_i_access_control},
            'openzeppelin/access/AccessControl.sol':{"content": oz_access_control},
            'openzeppelin/utils/Address.sol':{"content": oz_address},
            'openzeppelin/proxy/utils/Initializable.sol':{"content": oz_initializable},
            'openzeppelin/interfaces/draft-IERC1822.sol':{"content": oz_ierc1822},
            'openzeppelin/proxy/beacon/IBeacon.sol':{"content": oz_beacon_ibeacon},
            'openzeppelin/utils/StorageSlot.sol':{"content": oz_storage_slot},
            'openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol':{"content": oz_erc1967upgrade},
            'openzeppelin/proxy/utils/UUPSUpgradeable.sol':{"content": oz_uups_upgradeable},
            'SabotStakingV1.sol': {"content": sabot_staking}
        },
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version=_solc_version,
)
print(logic_contract)

SolcError: DeclarationError: Identifier not found or not unique.
   --> SabotStakingV1.sol:105:43:
    |
105 | contract SabotStakingV1 is Initializable, AccessControlUpgradeable, UUPSUpgradeable, SabotStakingBase{
    |                                           ^^^^^^^^^^^^^^^^^^^^^^^^


> command: `/Users/martinsmith/.solcx/solc-v0.8.9 --standard-json`
> return code: `0`
> stdout:
{"errors":[{"component":"general","errorCode":"7920","formattedMessage":"DeclarationError: Identifier not found or not unique.\n   --> SabotStakingV1.sol:105:43:\n    |\n105 | contract SabotStakingV1 is Initializable, AccessControlUpgradeable, UUPSUpgradeable, SabotStakingBase{\n    |                                           ^^^^^^^^^^^^^^^^^^^^^^^^\n\n","message":"Identifier not found or not unique.","severity":"error","sourceLocation":{"end":2945,"file":"SabotStakingV1.sol","start":2921},"type":"DeclarationError"}],"sources":{"SabotStakingV1.sol":{"id":0},"openzeppelin/access/AccessControl.sol":{"id":1},"